In [15]:
import torch.cuda

from src import  N_CLASSES, FEATURE_DIM

BATCH_SIZE = 100
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'device: {device}')

device: cpu


In [16]:
# 读数据
data = torch.load("data.pth")
feature = data['feature']
label = data['label']

In [17]:
# 生成训练集、测试集
from sklearn.model_selection import train_test_split

# x_train, x_test, y_train, y_test = train_test_split(feature, label, test_size=0.15, train_size=0.85, random_state=0)
# x_train, x_test, y_train, y_test = x_train.to(device), x_test.to(device), y_train.to(device), y_test.to(device)
x_train = feature.to(device)
y_train = label.to(device)

In [18]:
# 加载数据
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# test_dataset = TensorDataset(x_test, y_test)
# test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [19]:
# 定义网络
from model import Net
import torch.nn as nn

net = Net().to(device)

# 定义损失函数、优化器
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),
                       lr=0.001,
                       betas=(0.9, 0.999),
                       eps=1e-08,
                       weight_decay=0,
                       amsgrad=False)

In [20]:
# 训练网络
for epoch in range(200):  # 数字越大，循环次数越多
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    if epoch % 10 == 0:
        print(f'epoch {epoch} loss: {running_loss / 1500:.10f}')
        running_loss = 0.0

print('Finished Training')

# 训练结束后保存网络参数
PATH = './model.pth'
torch.save(net.state_dict(), PATH)

epoch 0 loss: 0.1156727313
epoch 10 loss: 0.0107814490
epoch 20 loss: 0.0061456303
epoch 30 loss: 0.0039568360
epoch 40 loss: 0.0034324323
epoch 50 loss: 0.0028770236
epoch 60 loss: 0.0022990988
epoch 70 loss: 0.0016972108
epoch 80 loss: 0.0016545222
epoch 90 loss: 0.0018570556
epoch 100 loss: 0.0014894591
epoch 110 loss: 0.0016068309
epoch 120 loss: 0.0011419157
epoch 130 loss: 0.0007955134
epoch 140 loss: 0.0012772046
epoch 150 loss: 0.0008910878
epoch 160 loss: 0.0008567379
epoch 170 loss: 0.0007111794
epoch 180 loss: 0.0006965153
epoch 190 loss: 0.0007300272
Finished Training


In [14]:
# 测试模型
net = Net().to(device)
net.load_state_dict(torch.load(PATH))

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    # noinspection PyShadowingBuiltins
    for input, labels in test_loader:
        # calculate outputs by running images through the network
        outputs = net(input)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        # noinspection PyUnresolvedReferences
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total}')

Accuracy: 0.9781111111111112
